In [1]:
import numpy as np 
import pandas as pd
from datetime import *
import glob

In [3]:
from seiscat import sfile_read

In [5]:
files=['../data/18-1333-00L.s201807']
event_cat,picks_cat = sfile_read.seisan_parsing(files)

events_df=pd.DataFrame(event_cat)
events_df

picks_df=pd.DataFrame(picks_cat)
picks_amp=picks_df[picks_df.Phase=='IAML']
picks_amp

NameError: name 'pd' is not defined